### Trabalhando com mapas interativos e mapas de calor
Para esse exemplo vamos utilizar a biblioteca **Folium**, que de forma simples, utiliza a API gratuita de mapas **Leaflet**. E a biblioteca **geopy** para, a partir de um endereço, localizar  a latitude e longitude para inserir pontos no mapa.

In [ ]:
# se ainda não possui a biblioteca Folium e Geopy, instale executando esse comando a partir do próprio Jupyter Notebook.
import sys
!{sys.executable} -m pip install folium geopy

In [1]:
#importando as bibliotecas que serão utilizadas
import epmwebapi as epm
from epmwebapi.dataobjectattributes import DataObjectAttributes
from epmwebapi.dataobjectsfilter import DataObjectsFilter
from epmwebapi.dataobjectsfilter import DataObjectsFilterType
from epmwebapi.domainfilter import DomainFilter
import datetime
import folium
from geopy.geocoders import Nominatim
import getpass
import pandas as pd

**Exemplo 1**

Adicionando localização, zoom inicial e marcador no mapa.

In [12]:
mapa = folium.Map(location=[-25.4284,-49.879873],zoom_start=7)
folium.Marker([-25.4284,-49.2733], popup='Curitiba').add_to(mapa)
mapa

**Exemplo 2**

Utilizando o Elipse DataModel foram criadas variáveis no EPM para armazenar a localização de objetos modelados no sistema SCADA(E3) que controla o sistema de ar condicionado das filiais da Elipse. O nome dessa variável é "Location".
Através da epmwebapi vamos localizar essas variáveis e mostrar no mapa a localização cadastrada. 


In [3]:
#Conexão ao EPM Server
user = input('EPM user:')
password = getpass.getpass("EPM password:")
#crie o objeto de conexão informando os endereços do EPM Webserver(Authentication Port e WEB API Port), usuário e senha.
connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', user, password)

EPM user:sa
EPM password:········


**Busca da variável "Location"**

Aqui faremos uma busca pela estrutura de dados criada pelo Elipse DataModel e retornaremos o valor da propriedade Location de cada uma das filiais.

In [4]:
basePath = '/Models/ElipseDataModel/DataModel/Elipse/'
filiais = ['[Elipse-RS]','[Elipse-PR]','[Elipse-SP]','[Elipse-MG]','[Elipse-RJ]']

locations = []
for filial in filiais:
    path = basePath + filial
    local = connection.getObjects([path])
    obj = list(local.values())[0]
    v = list(obj.enumProperties().values())[0]
    data = v.read()
    locations.append(data.value)
locations

['Rua 24 de Outubro, 353, Porto Alegre - RS, Brasil',
 'Av. Sete de Setembro, 4698, Curitiba - PR, Brasil',
 'Rua dos Pinheiros, 870, São Paulo - SP, Brasil',
 'Rua Antônio de Albuquerque, 156, Belo Horizonte - MG, Brasil',
 'Av. José Silva de Azevedo Neto, 200, Rio de Janeiro - RJ, Brasil']

**Convertendo o endereço em geolocalização usando a biblioteca geopy e adicionando marcadores ao mapa**

In [9]:
mapa = folium.Map(location=[-25.4284,-49.879873],zoom_start=5)
for local in locations:
    geolocator = Nominatim(user_agent="example_map")
    location = geolocator.geocode(local)
    print((location.latitude, location.longitude))
    folium.Marker([location.latitude,location.longitude]).add_to(mapa)
    
mapa  

(-30.0273301, -51.2039534)
(-25.4440481, -49.2839448)
(-23.5669455, -46.6862311)
(-19.9395084, -43.9302112)
(-22.9866285, -43.3603174)


**Exemplo 3 - Marcações especiais**

Inserindo uma tabela de valores no Popup do marcador. Clique para visualizar

In [22]:

#valores demonstração
df = pd.DataFrame(data=[['10:00','12.2', '1284'], ['12:00','15.6', '1740'],['14:00','16.0','1790']], columns=['Timestamp','WindSpeed', 'Power'])

html = df.to_html(classes='table table-striped table-hover table-condensed table-responsive')
popup = folium.Popup(html)

mapa = folium.Map(location=[37.20,140.02],zoom_start=11)

folium.Marker([37.20,140.02], popup=popup).add_to(mapa)
    
mapa 
